In [1]:
#内幕交易信息获取
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
data2 = {
'SType':'1',
'searchColumn': 'all',
'searchYear': 'all',
'preSWord':'' ,
'sword': '内幕交易',
'searchAgain':'' ,
'page': '3',
'res_wenzhong':'' ,
'res_wenzhonglist':'' ,
'wenzhong':'' ,
'pubwebsite': '/zjhpublic/'
}

In [3]:
log = []
def find_one_page(page):   #爬取一页数据
    data2['page'] = page
    r = requests.post('http://www.csrc.gov.cn/wcm/govsearch/simp_gov_list.jsp',data = data2)
    soup = BeautifulSoup(r.text, 'html.parser')
    allfind = soup.find_all('div','row')
    for row in allfind:
        onepage = []
        if ('中国证监会行政处罚决定书' in row.find('a').get_text()):
            onepage.append(row.find('a').get('href'))
            onepage.append(row.find('a').get_text())
            onepage.append(row.find('li','fbrq').get_text())
            onepage.append(row.find('li','wh').get_text())
            log.append(onepage)
    print ("page",page,"is ok")

In [4]:
log = []
for i in range(1,29):
    find_one_page(i)

page 1 is ok
page 2 is ok
page 3 is ok
page 4 is ok
page 5 is ok
page 6 is ok
page 7 is ok
page 8 is ok
page 9 is ok
page 10 is ok
page 11 is ok
page 12 is ok
page 13 is ok
page 14 is ok
page 15 is ok
page 16 is ok
page 17 is ok
page 18 is ok
page 19 is ok
page 20 is ok
page 21 is ok
page 22 is ok
page 23 is ok
page 24 is ok
page 25 is ok
page 26 is ok
page 27 is ok
page 28 is ok


In [8]:
out = pd.DataFrame(log,columns=['site','title','ann_date','number'])
out.to_csv('../data/site0.csv')

In [2]:
sitelist = pd.read_csv('../data/site0.csv',index_col = 0)

In [7]:
addressf = 'http://www.csrc.gov.cn'
log = []
num = 1
for add in sitelist['site']:
    address = addressf + add
    try:
        r = requests.get(address)
        r.encoding = 'utf - 8'
        soup = BeautifulSoup(r.text, 'html.parser')
        a = soup.find('div','mainContainer').get_text()
    except:
        r = requests.get(address[0:-13])
        r.encoding = 'utf - 8'
        soup = BeautifulSoup(r.text, 'html.parser')
        a = soup.find('div','mainContainer').get_text()
    log.append(a)
    print(num)
    num =num+1
    time.sleep(1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
out = pd.DataFrame(log,columns=['text'])
out.to_csv('../data/nmjy_text.csv')

In [9]:
nmjytext = pd.read_csv('../data/nmjy_text.csv',index_col = 0)

In [10]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
    return False

In [52]:
def get_nmjy_number(nmtext):    #验证一下至此没问题
    start = nmtext.find('20')
    end = nmtext.find('号')
    re = nmtext[start-1:end+1].replace('\xa0','')
    return re

num = 0
for t in nmjytext['text']:
    bj1 = get_nmjy_number(t)
    bj2 = sitelist['number'][num].replace(' ','')
    num = num + 1
    if (bj1 ==bj2):
        continue
    else:
        print(num)
        continue

48
359
362
377
384


In [63]:
def findnum(str0,postion):                 #找到该字符串的下一个数字
    try:
        if(is_number(str0[:postion])):
            return findnum(str0,postion+1)
        else:
            return postion
    except ValueError:
        pass
'''
def findwfsd(str1):
    current = str1.find('违法所得')
    str1 = str1[current+4:]
    currentpos = findnum(str1,1)-1
    if (currentpos>0):
        currentnum = str1[0:currentpos]
        return currentnum
    else:
        return findwfsd(str1)
'''

def findwfsd(str1):
    current = str1.find('违法所得')
    if(current != -1):
        str1 = str1[current+4:]
        currentpos = findnum(str1,1)-1
        if (currentpos>0):
            cend= str1.find('元')
            currentnum = str1[0:cend]
            return currentnum
        else:
            return findwfsd(str1)
    else:
        return('0')

In [66]:

log = []
index = 0
for i in nmjytext['text']:
    logs = []
    a = findwfsd(i)
    a= a.replace(',','')
    if(('万')in a):
        a = a.replace('余','')
        a = float(a.replace('万',''))*10000
    else:
        a = float(a.replace(',',''))
    if (a >= 1000000):
        logs.append(index)
        logs.append(a)     
        log.append(logs)
    index = index+1

In [ ]:
out = pd.DataFrame(log)
out.to_csv('../data/0wjb.csv')